In [ ]:
# links for training data, labels, and mapping
es_training = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/es_trial.text'
es_training_labels = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/es_trial.labels'
us_test = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_test.text'
es_test = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/es_test.text'
us_test_label = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_test.labels'
es_test_label = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/es_test.labels'
es_mapping = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/es_mapping.txt'
us_mapping = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_mapping.txt'

In [ ]:
def getListForLabel(df_train, num):
  """Returns a list of the tweets corresponding to a label
  
  :param df_train: dataframe to extract tweets from
  :param num: emoji label number
  :rtype: list
  :return: list consisting of tweets who have the label num
  """
  return df_train[df_train[df_train.columns[0]] == num].iloc[:, 1].values.tolist()

In [ ]:
def classVocab(train,vocab, vocab_class):
  """Returns number of words in a class
  
  :param train: list containing all tweets for a class
  :param vocab: dictionary of all unique words
  :param vocab_class: list of unique words for a class
  :rtype: int
  :return: int representing the number of words in the class
  """
  total_words_class = 0
  for sent in train:
    word_list = sent.split()
    for word in word_list:
      total_words_class+=1
      if word in vocab:
        vocab[word]+=1
      else:
        vocab[word]=1
      if word in vocab_class: #dictionary of class vocab
        vocab_class[word]+= 1
      else:
        vocab_class[word] = 1
  return total_words_class

In [ ]:
def trainData(training_data_df, num_classes): #replace str_i with i
  """Returns number of words in a class
  
  :param training_data_df: df containing all tweets and their corresponding labels
  :param num_classes: number of classes
  :rtype: dict
  :return: dict containing probability, vocab, and # of words in each class along with vocab size
  """
  training_data = {} #holds data for each class (eg. PositiveTrain or PositiveTest) 
  num_sentences_per_class = {} #number of sentences per class 
  class_probability = {} #class probability 
  vocab = {} #unique words (+counts)
  vocab_class = {} #nested dict #vocab words in each class 
  words_in_class = {} # # of total words in each class 
  total_num_sentences = 0 #total number of sentences in all classes combined
  
  for i in range(num_classes):
    training_data[i] = getListForLabel(training_data_df, i) #isolate rows based on label
    num_sentences_per_class[i] = len(training_data[i]) #number of sentences in the clasentences
    total_num_sentences += num_sentences_per_class[i] #add to total number of sentences
    vocab_class[i] = {}
    words_in_class[i] = classVocab(training_data[i], vocab, vocab_class[i])
  
  vocab_size = len(vocab.keys()) #number of unique words 

  for i in range(num_classes):
    class_probability[i] = num_sentences_per_class[i]/total_num_sentences #probability of a class
 #probability of a class
  
  return {'class prob' : class_probability, 'vocab class': vocab_class, 'words_in_class' : words_in_class, 'vocab size': vocab_size}

In [ ]:
def prediction(test_dict, prob_class, vocab_class, total_words_class, vocab_size, num_classes, alpha):
  """Returns list with predictions
  
  :param test_dict: dict containing tweets for each class; key = class, value = list of tweets
  :param prob_class: dict containing probability of each class; key = class, value = probability of class
  :param vocab_class: dict containing vocab words for each class; key = class, value = list of words in the class
  :param total_words_class: dict containing number of words for each class; key = class, value = # of words in the class
  :param vocab_size: number of vocab words
  :param num_classes: number of classes
  :param alpha: alpha value
  :rtype: list
  :return: list containing tuples of sentences with their actual and predicted labels
  """
  prediction_list = []
  for i in range(num_classes):
    for sent in test_dict[i]:
      prob_list = []
      word_list = sent.split()
      actual_sentiment = i
      for j in range(num_classes):
        prob_list.append(find_prob(word_list,prob_class[j], vocab_class[j], total_words_class[j], vocab_size, alpha))
      pred_sentiment = prob_list.index(max(prob_list))
      pred=(sent, actual_sentiment, pred_sentiment)
      prediction_list.append(pred)
  return prediction_list


In [ ]:
def find_prob(list_words, prob_class, vocab_class, total_words_class, total_words, alpha):
  """Returns list with predictions
  
  :param list_words: list of words in a sentence
  :param prob_class: probability of the class
  :param vocab_class: vocab words in the class
  :param total_words_class: number of words in the class
  :param total_words: total number of words
  :param alpha: alpha value
  :rtype: float
  :return: probability that the sentence is in a certain class
  """
  sent_prob = prob_class
  for word in list_words:
    if word in vocab_class:
      numerator = vocab_class[word] + alpha
    else:
      numerator = alpha
    denominator = total_words_class + alpha* total_words
    word_prob = numerator / denominator
    sent_prob = sent_prob * word_prob
  return sent_prob

In [ ]:
def test(prediction_list):
  """Prints accuracy of predictions

  :param prediction_list: list of predictions
  """
  correct = 0
  total_test_sample = 0
  for pred in prediction_list:
    total_test_sample +=1 
    (sent, actual_sentiment, pred_sentiment) = pred
    if actual_sentiment == pred_sentiment:
      correct +=1  
  accuracy = correct/total_test_sample
  print("accuracy: ", accuracy)

In [ ]:
import pandas as pd
def getData(text_url, label_url):
  """Returns df with the tweets and their corresponding labels, and the number of classes

  :param text_url: url containing tweets
  :param label_url: url containing labels for the tweets
  :rtype: df, int
  :return: df with tweets+labels, and number of classes
  """
  df =  pd.read_fwf(us_training, header = None)
  labels = pd.read_csv(label_url, delimiter = "\n", header = None, error_bad_lines=False)

  data = pd.concat([labels, df], axis = 1, ignore_index=True)
  data = data.dropna(axis='columns')
  return data, len(pd.unique(data.iloc[:, 0]))

In [ ]:
#MAIN METHOD#
import pandas as pd
us_training = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_trial.text'
us_training_labels = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_trial.labels'
us_test = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_test.text'
us_test_labels = 'https://raw.githubusercontent.com/RobertWetzler/CSE3521_EmojiPrediction/main/us_test.labels'

us_training_data, num_classes = getData(us_training, us_training_labels)
combo_dict = trainData(us_training_data, num_classes)

test_data = {}
us_test_data, num_classes = getData(us_test, us_test_labels)
for i in range(num_classes):
    test_data[i] = getListForLabel(us_test_data, i) #isolate rows based on label
predictions = prediction(test_data, combo_dict['class prob'], combo_dict['vocab class'], combo_dict['words_in_class'], combo_dict['vocab size'], num_classes, 10)
test(predictions)


accuracy:  0.2123


BELOW:
WIP; taking input tweet from user and outputting the predicted emoji


In [ ]:
emojis = {
0:	'❤',
1:	'😍',
2:	'😂',
3:	'💕',
4:	'🔥',
5:	'😊',	
6:	'😎',
7:	'✨',
8:	'💙',	
9:	'😘',
10:	'📷',
11:	'🇺🇸',
12:	'☀',
13:	'💜',
14:	'😉',	
15:	'💯',	
16:	'😁',
17:	'🎄',	
18:	'📸',
19:	'😜'
}
text = "."
while len(text) > 0:
  text = input("Enter your tweet: ")
  # predict somehow
  # print predicted emoji

Enter your tweet: 
